# Using FABRIC Persistent Storage

Your compute nodes can include persistent Project specific storage volumes.
These volumes can be created on request at various sites through
the (Help Portal)[https://fabric-testbed.atlassian.net/servicedesk/customer/portal/2/group/12].

This example notebook will demonstrate how to attach a persistent storage to a compute node in your slice.


## Setup the Experiment

#### Import FABRIC API

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
fablib.show_config()


### Load the plugins

In [ ]:
import traceback
from plugins import Plugins
try:
    Plugins.load()
except Exception as e:
    traceback.print_exc()

## Create a Node

The cell below creates a slice that contains a single node with storage volume created out of scope of the slice.

In [ ]:
# ### Set the Slice Name and FABRIC Site
slice_name = "MySlice-storage"
node_name = 'Node1'
storage_name = 'vol1'

In [ ]:
try:
    # Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Add node
    node = slice.add_node(name=node_name, site="RENC")

    # Add an NVME Drive
    node.add_storage(name=storage_name)

    # Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Get the Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Get the Node

Retrieve the storage device name and mount it


In [ ]:
try:
    node = slice.get_node(node_name)
    print(f"{node}")
    storage = node.get_storage(storage_name)
    print(f"{storage}")
    print(f"Storage Device Name: {storage.get_device_name()}")
    print("Mounting the storage volume")
    stdout, stderr = node.execute(f"sudo mkdir -p /mnt/{storage_name};"
                                  f"sudo mount {storage.get_device_name()} /mnt/{storage_name}")
    print(stdout)
    print(stderr)
except Exception as e:
    print(f"Exception: {e}")

## Cleanup Your Experiment

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")